In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
import requests
import pickle
import numpy as np
from pathlib import Path
from datetime import datetime

from src.nlp.generate_lyrics import DeepLyric
from src.nlp.evaluate_lyrics import Evaluator

## Model Loading

The `DeepLyric` object can now be loaded by two different methods.
1. Load the model object and itos object directly
2. Input model `str` and download from cloud storage

### Load Method 1
It's important to input `model_name` explicitly if this method is used

In [3]:
MODEL = '4.2-LM-108k-lines-genre-song_title'

In [4]:
model_url = f'https://storage.googleapis.com/w210-capstone/models/{MODEL}_architecture.pkl'
model = requests.get(model_url)
model = model.content
model = pickle.loads(model)

In [5]:
itos_url = f'https://storage.googleapis.com/w210-capstone/models/{MODEL}_itos.pkl'
itos = requests.get(itos_url)
itos = itos.content
itos = pickle.loads(itos)

In [55]:
deep_lyric = DeepLyric(model, itos=itos, model_type='language', model_name=MODEL)

In [56]:
deep_lyric.config

{'seed_text': 'xbos',
 'max_len': 40,
 'GPU': True,
 'context_length': 30,
 'beam_width': 3,
 'verbose': 0,
 'temperature': 1.5,
 'top_k': 3,
 'audio': None,
 'multinomial': True,
 'model_name': '4.2-LM-108k-lines-genre-song_title',
 'model_type': 'language'}

### Load Method 2

This method is more convenient, but takes a few extra seconds at the beginning to download the models

In [61]:
deep_lyric = DeepLyric(MODEL, model_type='language')

In [62]:
deep_lyric.config

{'seed_text': 'xbos xgenre death metal xtitle',
 'max_len': 40,
 'GPU': True,
 'context_length': 40,
 'beam_width': 3,
 'verbose': 0,
 'temperature': 1.5,
 'top_k': 5,
 'audio': None,
 'multinomial': True,
 'model_name': '4.2-LM-108k-lines-genre-song_title',
 'model_type': 'language'}

## Changing Parameters, Generating, and Saving
Once the model is loaded, the rest of the functions work identically. There are also two ways to set configs.
1. Pass in a key/value pair
2. Pass in an entire dictionary (this will overwrite all current config)

In [57]:
deep_lyric.set_config('seed_text', 'xbos xgenre death metal xtitle')
deep_lyric.set_config('max_len', 40)
deep_lyric.set_config('top_k', 5)
deep_lyric.set_config('context_length', 40)
deep_lyric.set_config('verbose', 0)

In [58]:
deep_lyric.generate_text()

Here is the printed lyrics from the above instance

In [59]:
deep_lyric.print_lyrics()

SONG START
 genre: death metal 
 title: slaves of everything 

xbol-1 is there any difference ? 
 how do you see that we are all dead ? 
 is it all in everyone ? 
 see we don 't love anyone 
 you 

Here's a preview of the json payload. Simply change the parameter `dir` to a directory to save to disk.
File names are UTC timestamps to attempt to be unique

In [60]:
deep_lyric.save_json(out=True)

{'meta': {'seed_text': 'xbos xgenre death metal xtitle',
  'max_len': 40,
  'GPU': True,
  'context_length': 40,
  'beam_width': 3,
  'verbose': 0,
  'temperature': 1.5,
  'top_k': 5,
  'audio': None,
  'multinomial': True,
  'model_name': '4.2-LM-108k-lines-genre-song_title',
  'model_type': 'language'},
 'lyric': ['xbos',
  'xgenre',
  'death',
  'metal',
  'xtitle',
  'slaves',
  'of',
  'everything',
  'xbol-1',
  'is',
  'there',
  'any',
  'difference',
  '?',
  'xeol',
  'xbol-2',
  'how',
  'do',
  'you',
  'see',
  'that',
  'we',
  'are',
  'all',
  'dead',
  '?',
  'xeol',
  'xbol-3',
  'is',
  'it',
  'all',
  'in',
  'everyone',
  '?',
  'xeol',
  'xbol-4',
  'see',
  'we',
  'don',
  "'t",
  'love',
  'anyone',
  'xeol',
  'xbol-5',
  'you']}

And if we wanted to run a bunch of simulations with the specified parameters, we just do this:

In [ ]:
# for i in range(10):
#     deep_lyric.generate_text()
#     deep_lyric.save_json(dir='.')

## Evaluation

The `Evaluator` is an extension of `DeepLyric` that piggy-backs off the config states. We can pass the configured deep lyric object above to do further evals

None of the actual eval functions are hooked up yet, but here is how we'd interface with the framework

In [61]:
# evaluator = Evaluator(deep_lyric, set_lyric_state=False)
evaluator = Evaluator(deep_lyric)

In [63]:
evaluator.deep_lyric.config

{'seed_text': 'xbos xgenre death metal xtitle',
 'max_len': 40,
 'GPU': True,
 'context_length': 40,
 'beam_width': 3,
 'verbose': 0,
 'temperature': 1.5,
 'top_k': 5,
 'audio': None,
 'multinomial': True,
 'model_name': '4.2-LM-108k-lines-genre-song_title',
 'model_type': 'language'}

In [75]:
# evaluator.get_lyric()
# evaluator.generated_song

In [74]:
# not yet implemented
evaluator.get_rhyme_density()
evaluator.metrics

{'metric1': None,
 'metric2': None,
 'metric3': None,
 'rhyme_density_a': 'gloom',
 'rhyme_density_b': 'and'}

In [109]:
# evaluator.save_json(out=True)

In [106]:
evaluator.metrics

{'metric1': None,
 'metric2': None,
 'metric3': None,
 'rhyme_density_a': 'gloom',
 'rhyme_density_b': 'and'}

In [110]:
# from pandas.io.json import json_normalize
# import json
# json_normalize(evaluator.deep_lyric.config, errors='ignore')

### Store to Cloud...

In [94]:
OUT_STORE = f'gs://w210-capstone/lyrics/{model_name}/'

In [95]:
!gsutil -m -o cp $OUT_PATH/* $OUT_STORE

Copying file://../data/lyrics/4.2-LM-108k-lines-genre-song_title/1543790822-1.0-1.txt [Content-Type=text/plain]...
Copying file://../data/lyrics/4.2-LM-108k-lines-genre-song_title/1543790825-1.0-2.txt [Content-Type=text/plain]...
Copying file://../data/lyrics/4.2-LM-108k-lines-genre-song_title/1543790834-1.0-3.txt [Content-Type=text/plain]...
Copying file://../data/lyrics/4.2-LM-108k-lines-genre-song_title/1543790842-1.0-4.txt [Content-Type=text/plain]...
- [4 files][  2.7 KiB/  2.7 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m -o ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://../data/lyrics/4.2-LM-108k-lines-genre-song_title/1543790850-1.0-5.txt [Content-Type=text/plain]...
Copying file://../data/lyrics/4.2-LM-108k-lines-genre-song_title/1543790859-1.2-1.txt [Conte